In [1]:
import numpy as np
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import glob
import cv2
import torch
from sklearn.cluster import KMeans
import os

In [2]:
import json

def read_poses_from_json(file_path):
    """Reads the pose data from a JSON file."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def write_matrices_to_txt(data, output_file_path):
    """Writes the 4x4 pose matrices to a text file."""
    with open(output_file_path, 'w') as file:
        for frame_id, frame_data in data.items():
            pose = frame_data['pose']
            #file.write(f"{frame_id}:\n")
            for row in pose:
                formatted_row = ' '.join(f"{value:.18e}" for value in row)
                file.write(formatted_row + "\n")

def write_intirinsic_to_txt(data, output_file_path):
    """Writes the 4x4 pose matrices to a text file."""
    with open(output_file_path, 'w') as file:
        for frame_id, frame_data in data.items():
            pose = frame_data['intrinsic']
            #file.write(f"{frame_id}:\n")
            for row in pose:
                formatted_row = ' '.join(f"{value:.18e}" for value in row)
                file.write(formatted_row + "\n")



In [3]:

# Paths to your JSON input and text output files
input_json_path = '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/pose_intrinsic_imu.json'  # Change this to the path of your JSON file
output_txt_path = '/home/rozenberszki/D_Project/wsnsl/Dataset/traj.txt'  # Change this to your desired output file path

# Read the data from JSON
pose_data = read_poses_from_json(input_json_path)

# Write the matrices to a text file
write_matrices_to_txt(pose_data, output_txt_path)


pose_data = read_poses_from_json(input_json_path)
output_txt_path = '/home/rozenberszki/D_Project/wsnsl/Dataset/intrinsic.txt'
# Write the matrices to a text file
write_intirinsic_to_txt(pose_data, output_txt_path)


In [4]:
import torch.nn.functional as F
def readDepth(filepath):
    depth = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
    depth_data = depth.astype(np.float32) / 6553.5
    depth_data = torch.from_numpy(depth_data)
    #print(depth_data.shape)
    #print(depth_data[0,0])
    return depth_data

In [5]:

color_paths = sorted(glob.glob(f"/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/*.jpg"))
print(color_paths)

color_data = cv2.imread(color_paths[0])
image = cv2.cvtColor(color_data, cv2.COLOR_BGR2RGB)

depth_paths = sorted(glob.glob(f"/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/processedData/depth/*.png"))
print("image",image.shape)
print("DEPT",readDepth(depth_paths[0]).shape)

['/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000000.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000001.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000002.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000003.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000004.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000005.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000006.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000007.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000008.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000009.jpg', '/home/ro

In [6]:
import glob
from tqdm import tqdm  # Import tqdm

def readColorImage(filepath):
    # Read color image and convert to RGB (OpenCV uses BGR by default)
    image = cv2.imread(filepath, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convert to float32 and normalize to [0, 1] if needed
    return torch.from_numpy(image.astype(np.float32) / 255.0)

def save_depth_image(input_path, output_filepath,amount=0, upscale=1):
    """Saves the depth image to a file."""
    depth_paths = sorted(
            glob.glob(os.path.join(input_path, "*.png"))
        )
    #print(depth_paths)
    os.makedirs(output_filepath, exist_ok=True)  # Ensure output path exists
    depth_paths=depth_paths[:amount]
    for i, depth_path in enumerate(tqdm(depth_paths, desc='Upsampling depth images')):
        # this changes the vals
        depth_data=readDepth(depth_path)*upscale
        # this image shape
        upsampled_depth_bilinear = F.interpolate(depth_data.unsqueeze(0).unsqueeze(0), 
                                                scale_factor=upscale, 
                                                mode='nearest', 
                                               ).squeeze()
        #print(upsampled_depth_bilinear.shape)
        upsampled_depth_bilinear = upsampled_depth_bilinear
        upsampled_depth = upsampled_depth_bilinear * 6553.5
        upsampled_depth = upsampled_depth.numpy().astype(np.uint16)
        #print((upsampled_depth.shape))

        cv2.imwrite(os.path.join(output_filepath, f"frame_{str(i).zfill(6)}.png"), upsampled_depth)
        
#bilateral filter
def save_color_image(input_path, output_filepath, amount=0, upscale=1):
    """Saves the color images to a file after upscaling."""
    color_paths = sorted(glob.glob(os.path.join(input_path, "*.jpg")))
    color_paths = color_paths[:amount] if amount > 0 else color_paths
    
    os.makedirs(output_filepath, exist_ok=True)  # Ensure output path exists
    
    for i, color_path in enumerate(tqdm(color_paths, desc='Processing color images')):
        color_data = readColorImage(color_path)
        color_data = color_data.permute(2, 0, 1).unsqueeze(0)  # CxHxW and add batch dimension
        
        upsampled_color_bilinear = F.interpolate(color_data, 
                                                scale_factor=upscale, 
                                                mode='nearest', 
                                                ).squeeze(0)
        
        # Convert back to uint8 and BGR for saving
        upsampled_color = (upsampled_color_bilinear.permute(1, 2, 0) * 255).byte().numpy()
        upsampled_color = cv2.cvtColor(upsampled_color, cv2.COLOR_RGB2BGR)
        #print(upsampled_color.shape)
        cv2.imwrite(os.path.join(output_filepath, f"frame_{str(i).zfill(6)}.jpg"), upsampled_color)

save_color_image('/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/',amount=2000,upscale=0.5)
save_depth_image('/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/depth', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/',amount=2000,upscale=3.75)
#0.5, 3.75

Upsampling depth images:  77%|███████▋  | 1540/2000 [00:16<00:04, 111.69it/s]

In [2]:

def pose_matrix_from_quaternion( pvec):
    """convert 4x4 pose matrix to (t, q)"""
    from scipy.spatial.transform import Rotation

    pose = np.eye(4)
    pose[:3, :3] = Rotation.from_quat(pvec[3:]).as_matrix()
    pose[:3, 3] = pvec[:3]
    print(pose)
    return pose

In [3]:
def parse_list( filepath, skiprows=0):
    """read list data"""
    data = np.loadtxt(filepath, delimiter=" ", dtype=np.unicode_, skiprows=skiprows)
    return data


In [9]:
pose_list="/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/colmap/images.txt"
pose_data = parse_list(pose_list, skiprows=0)
print(pose_data.shape)
print(pose_data[0,1:-1])
pose_data_temp = np.zeros((pose_data.shape[0], 4,4))
for i in range(pose_data.shape[0]):
    pose_data_temp[i] = pose_matrix_from_quaternion(pose_data[i,1:-2])
output_txt_path = '/home/rozenberszki/D_Project/wsnsl/Dataset/pose.txt'
np.savetxt(output_txt_path, pose_data_temp.reshape(-1, pose_data_temp.shape[-1]), fmt='%f')

(946, 10)
['0.350173' '0.929442' '-0.101077' '0.0574443' '-0.413599' '1.81707'
 '0.290624' '1']
[[-0.95070331 -0.30997459 -0.00888705  0.350173  ]
 [ 0.28328346 -0.85646879 -0.43152254  0.929442  ]
 [ 0.12614954 -0.41276746  0.90205838 -0.101077  ]
 [ 0.          0.          0.          1.        ]]
[[-0.94288029 -0.33303966 -0.0078325   0.34196   ]
 [ 0.30601197 -0.85659029 -0.4154633   0.931543  ]
 [ 0.13165651 -0.39412899  0.90957622 -0.107738  ]
 [ 0.          0.          0.          1.        ]]
[[-0.93135321 -0.36381636 -0.01479395  0.332623  ]
 [ 0.33900143 -0.85156735 -0.39988883  0.934831  ]
 [ 0.13288805 -0.37745291  0.91644425 -0.109968  ]
 [ 0.          0.          0.          1.        ]]
[[-0.92467156 -0.37979197 -0.02721331  0.329513  ]
 [ 0.35967109 -0.84775249 -0.38981076  0.937023  ]
 [ 0.12497684 -0.37023477  0.92049281 -0.105328  ]
 [ 0.          0.          0.          1.        ]]
[[-0.93293394 -0.3593917  -0.02172249  0.335129  ]
 [ 0.34150586 -0.86416627 -0.3695

In [10]:
np.linalg.inv(pose_data_temp[0])

array([[-0.95070331,  0.28328346,  0.12614954,  0.0823659 ],
       [-0.30997459, -0.85646879, -0.41276746,  0.8628615 ],
       [-0.00888705, -0.43152254,  0.90205838,  0.49536453],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])